# Embedded CFG Experiment `gpiote`

Testing the embedded FXE implementation on `gpiote` example from the nRR5 SDK (v17.0.2)

In [1]:
import os
from copy import deepcopy

from efxe import *

In [2]:
%%time
gpiote_efxe = EmFXEngine(
    pd="mmaps/nrf52832.yml",
    fw_path="examples/gpiote.elf",
    log_stdout=True,
    log_insn=True,
)
gpiote_efxe.run()

2022-11-19 20:00:52,430 : backup context @ 0x2b1
2022-11-19 20:00:52,431 : backup context @ 0x2b1
2022-11-19 20:00:52,431 : backup context @ 0x2b1
2022-11-19 20:00:52,432 : backup context @ 0x2b1
2022-11-19 20:00:52,432 : backup context @ 0x2b1
2022-11-19 20:00:52,432 : backup context @ 0x2b1
2022-11-19 20:00:52,433 : backup context @ 0x2b1
2022-11-19 20:00:52,433 : backup context @ 0x2b1
2022-11-19 20:00:52,433 : backup context @ 0x2b1
2022-11-19 20:00:52,434 : backup context @ 0x2b1
2022-11-19 20:00:52,434 : backup context @ 0x2b1
2022-11-19 20:00:52,434 : backup context @ 0x2b1
2022-11-19 20:00:52,435 : backup context @ 0x2b1
2022-11-19 20:00:52,435 : backup context @ 0x2b1
2022-11-19 20:00:52,435 : backup context @ 0x2b1
2022-11-19 20:00:52,436 : backup context @ 0x2b1
2022-11-19 20:00:52,436 : backup context @ 0x2b1
2022-11-19 20:00:52,436 : backup context @ 0x2b1
2022-11-19 20:00:52,437 : backup context @ 0x2b1
2022-11-19 20:00:52,437 : backup context @ 0x2b1
2022-11-19 20:00:52,

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



2022-11-19 20:00:54,738 : 0xe6a: f06f 0001  mvn.w	r0, #1
2022-11-19 20:00:54,739 : 0xe6e: f8c1 0400  str.w	r0, [r1, #1024]	; 0x400
2022-11-19 20:00:54,739 : pc @ 0x00000E6E : w 0xfffffffe @ 0x40000400
2022-11-19 20:00:54,739 : 0xe72: 2a06       cmp	r2, #6
2022-11-19 20:00:54,739 : 0xe74: f8d3 3134  ldr.w	r3, [r3, #308]	; 0x134
2022-11-19 20:00:54,739 : pc @ 0x00000E74 : r 0x00000000 @ 0x10000134
2022-11-19 20:00:54,740 : 0xe78: f43f af7a  beq.w	d70 <SystemInit+0x1d8>
2022-11-19 20:00:54,740 : backup context @ 0xe79
2022-11-19 20:00:54,740 : new basic block @ 0xe7c
2022-11-19 20:00:54,740 : 0xe7c: e6ed       b.n	c5a <SystemInit+0xc2>
2022-11-19 20:00:54,741 : old block writes mem @ 0xc5a
2022-11-19 20:00:54,741 : 0xc5a: 4aa2       ldr	r2, [pc, #648]	; (ee4 <SystemInit+0x34c>)
2022-11-19 20:00:54,741 : pc @ 0x00000C64 : w 0x00f00000 @ 0xE000ED88
2022-11-19 20:00:54,741 : restore context @ 0xd71
2022-11-19 20:00:54,742 : new basic block @ 0xd70
2022-11-19 20:00:54,742 : 0xd70: 3b03       

UcError: Invalid memory write (UC_ERR_WRITE_UNMAPPED)

In [3]:
total_insns = 0
total_size = 0
for block in gpiote_efxe.bblocks.values():
    total_size += block.size
    for insn in gpiote_efxe.cs.disasm(gpiote_efxe.uc.mem_read(block.addr, block.size), block.size):
        total_insns += 1
print(f"Found {len(gpiote_efxe.bblocks)} blocks, {total_insns} insns, {hex(total_size)} bytes\n")
# for k, v in gpiote_efxe.bblocks.items():
#     print(hex(k), v)

Found 375 blocks, 1723 insns, 0x12c6 bytes



In [4]:
disasm_txt = deepcopy(gpiote_efxe.fw.disasm_txt)
for block in gpiote_efxe.bblocks.values():
    insn_addr = block.addr
    for insn in gpiote_efxe.cs.disasm(
            gpiote_efxe.uc.mem_read(block.addr, block.size), block.size):
        try:
            lineno = gpiote_efxe.fw.disasm[insn_addr]['line']
            insn_addr += insn.size
            disasm_txt[lineno] = "{}{}{}{:<100}\u001b[0m".format(
                "\u001b[103m",  # bright yellow background
                "\u001b[30m",   # black foreground
                "\u001b[1m",    # bold font
                disasm_txt[lineno])
        except Exception:
            pass
print('\n'.join(disasm_txt))


examples/gpiote.elf:     file format elf32-littlearm


Disassembly of section .text:

00000000 <__isr_vector>:
       0:	20010000 	.word	0x20010000
       4:	000002b1 	.word	0x000002b1
       8:	000002d9 	.word	0x000002d9
       c:	000002db 	.word	0x000002db
      10:	000002dd 	.word	0x000002dd
      14:	000002df 	.word	0x000002df
      18:	000002e1 	.word	0x000002e1
	...
      2c:	000002e3 	.word	0x000002e3
      30:	000002e5 	.word	0x000002e5
      34:	00000000 	.word	0x00000000
      38:	000002e7 	.word	0x000002e7
      3c:	000002e9 	.word	0x000002e9
      40:	000002eb 	.word	0x000002eb
      44:	000002eb 	.word	0x000002eb
      48:	000002eb 	.word	0x000002eb
      4c:	000002eb 	.word	0x000002eb
      50:	000002eb 	.word	0x000002eb
      54:	000002eb 	.word	0x000002eb
      58:	00000575 	.word	0x00000575
      5c:	000002eb 	.word	0x000002eb
      60:	00000a71 	.word	0x00000a71
      64:	000002eb 	.word	0x000002eb
      68:	000002eb 	.word	0x000002eb
      6c:	000002eb 	.word	0x0000

In [5]:
import os
from copy import deepcopy

from efxe import *

In [6]:
%%time
stimer_efxe = EmFXEngine(
    pd="mmaps/nrf52832.yml",
    fw_path="examples/simple_timer.elf",
    log_stdout=True,
    log_insn=True,
)
stimer_efxe.run()

2022-11-19 20:00:55,105 : backup context @ 0x2b1
2022-11-19 20:00:55,105 : backup context @ 0x2b1
2022-11-19 20:00:55,107 : backup context @ 0x2b1
2022-11-19 20:00:55,107 : backup context @ 0x2b1
2022-11-19 20:00:55,107 : backup context @ 0x2b1
2022-11-19 20:00:55,107 : backup context @ 0x2b1
2022-11-19 20:00:55,108 : backup context @ 0x2b1
2022-11-19 20:00:55,108 : backup context @ 0x2b1
2022-11-19 20:00:55,108 : backup context @ 0x2b1
2022-11-19 20:00:55,108 : backup context @ 0x2b1
2022-11-19 20:00:55,108 : backup context @ 0x2b1
2022-11-19 20:00:55,108 : backup context @ 0x2b1
2022-11-19 20:00:55,109 : backup context @ 0x2b1
2022-11-19 20:00:55,109 : backup context @ 0x2b1
2022-11-19 20:00:55,109 : backup context @ 0x2b1
2022-11-19 20:00:55,109 : backup context @ 0x2b1
2022-11-19 20:00:55,110 : backup context @ 0x2b1
2022-11-19 20:00:55,110 : backup context @ 0x2b1
2022-11-19 20:00:55,110 : backup context @ 0x2b1
2022-11-19 20:00:55,110 : backup context @ 0x2b1
2022-11-19 20:00:55,

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



2022-11-19 20:00:57,377 : 0x3e4: 4a04       ldr	r2, [pc, #16]	; (3f8 <app_simple_timer_event_handler+0x28>)
2022-11-19 20:00:57,377 : pc @ 0x000003E4 : r 0x20000034 @ 0x000003F8
2022-11-19 20:00:57,377 : pc @ 0x000003E4 : r 0x20000034 @ 0x000003F8
2022-11-19 20:00:57,378 : backup context @ 0x3e5
2022-11-19 20:00:57,378 : backup context @ 0x3e5
2022-11-19 20:00:57,378 : PC @ 0x3e4 : volatile r 0x20000034 @ 0x000003F8
2022-11-19 20:00:57,378 : PC @ 0x3e4 : volatile r 0x20000034 @ 0x000003F8
2022-11-19 20:00:57,379 : pc @ 0x000003E4 : r 0x       0 @ 0x000003F8
2022-11-19 20:00:57,379 : pc @ 0x000003E4 : r 0x       0 @ 0x000003F8
2022-11-19 20:00:57,379 : backup context @ 0x3e5
2022-11-19 20:00:57,379 : backup context @ 0x3e5
2022-11-19 20:00:57,379 : PC @ 0x3e4 : volatile r 0x00000000 @ 0x000003F8
2022-11-19 20:00:57,379 : PC @ 0x3e4 : volatile r 0x00000000 @ 0x000003F8
2022-11-19 20:00:57,380 : pc @ 0x000003E4 : r 0x20000034 @ 0x000003F8
2022-11-19 20:00:57,380 : pc @ 0x000003E4 : r 0x20

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [7]:
for addr, data in sorted(stimer_efxe.volatile_addrs.items()):
    print("{:>11}  Writes: {}".format(
        hex(addr) + ':', 
        ' '.join([hex(v) for v in data['writes']]),
    ))
    for i, context in enumerate(data['contexts']):
        print("{:>11}  <Context PC@{:08X} | irq {:d} | 0x20000030:{:08X}>".format(
            '',
            context.pc,
            context.vtentry,
            int.from_bytes(context.mem_state[0x20000030], 'little') \
                if 0x20000030 in context.mem_state else 0,
        ))

     0x300:  Writes: 
             <Context PC@000002ED | irq 100 | 0x20000030:00000B4D>
             <Context PC@000002ED | irq 100 | 0x20000030:00000B4D>
     0x350:  Writes: 
             <Context PC@00000331 | irq 100 | 0x20000030:00000B4D>
             <Context PC@00000331 | irq 100 | 0x20000030:00000B4D>
             <Context PC@00000331 | irq 100 | 0x20000030:00000B4D>
             <Context PC@00000331 | irq 100 | 0x20000030:00000B4D>
             <Context PC@00000331 | irq 100 | 0x20000030:00000B4D>
             <Context PC@00000331 | irq 100 | 0x20000030:00000B4D>
             <Context PC@00000331 | irq 0 | 0x20000030:00000000>
     0x3f0:  Writes: 
             <Context PC@000003D9 | irq 100 | 0x20000030:00000B4D>
     0x3f4:  Writes: 
             <Context PC@000003DF | irq 100 | 0x20000030:00000B4D>
     0x3f8:  Writes: 
             <Context PC@000003E5 | irq 100 | 0x20000030:00000B4D>
             <Context PC@000003E5 | irq 100 | 0x20000030:00000B4D>
             <Context

In [8]:
total_insns = 0
total_size = 0
for block in stimer_efxe.bblocks.values():
    total_size += block.size
    for insn in stimer_efxe.cs.disasm(
            stimer_efxe.uc.mem_read(block.addr, block.size), block.size):
        total_insns += 1
print(f"Found {len(stimer_efxe.bblocks)} blocks, {total_insns} insns, {hex(total_size)} bytes\n")
# for k, v in gpiote_efxe.bblocks.items():
#     print(hex(k), v)

Found 404 blocks, 1769 insns, 0x131e bytes



In [9]:
disasm_txt = deepcopy(stimer_efxe.fw.disasm_txt)
for block in stimer_efxe.bblocks.values():
    insn_addr = block.addr
    for insn in stimer_efxe.cs.disasm(
            stimer_efxe.uc.mem_read(block.addr, block.size), block.size):
        try:
            lineno = stimer_efxe.fw.disasm[insn_addr]['line']
            insn_addr += insn.size
            disasm_txt[lineno] = "{}{}{}{:<100}\u001b[0m".format(
                "\u001b[103m",  # bright yellow background
                "\u001b[30m",   # black foreground
                "\u001b[1m",    # bold font
                disasm_txt[lineno])
        except Exception:
            pass
print('\n'.join(disasm_txt))


examples/simple_timer.elf:     file format elf32-littlearm


Disassembly of section .text:

00000000 <__isr_vector>:
       0:	20010000 	.word	0x20010000
       4:	000002b1 	.word	0x000002b1
       8:	000002d9 	.word	0x000002d9
       c:	000002db 	.word	0x000002db
      10:	000002dd 	.word	0x000002dd
      14:	000002df 	.word	0x000002df
      18:	000002e1 	.word	0x000002e1
	...
      2c:	000002e3 	.word	0x000002e3
      30:	000002e5 	.word	0x000002e5
      34:	00000000 	.word	0x00000000
      38:	000002e7 	.word	0x000002e7
      3c:	000002e9 	.word	0x000002e9
      40:	000002eb 	.word	0x000002eb
      44:	000002eb 	.word	0x000002eb
      48:	000002eb 	.word	0x000002eb
      4c:	000002eb 	.word	0x000002eb
      50:	000002eb 	.word	0x000002eb
      54:	000002eb 	.word	0x000002eb
      58:	00000515 	.word	0x00000515
      5c:	000002eb 	.word	0x000002eb
      60:	000002eb 	.word	0x000002eb
      64:	00000af5 	.word	0x00000af5
      68:	000002eb 	.word	0x000002eb
      6c:	000002eb 	.word	